## Import requirements

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from pandas.errors import MergeError

/tmp/ipykernel_22104/1508799565.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Data Preparation

### Dataset understanding

In [2]:
train = pd.read_csv('/home/mehran/Projects/horse_survival/train.csv')
test = pd.read_csv('/home/mehran/Projects/horse_survival/test.csv')

In [3]:
train.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,...,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,...,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,...,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,...,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,...,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


In [4]:
test.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1235,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,...,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,1236,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,...,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,1237,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,...,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,1238,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,...,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,1239,yes,adult,529840,38.3,50.0,12.0,NaN,normal,bright_pink,...,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1235 non-null   int64  
 1   surgery                1235 non-null   object 
 2   age                    1235 non-null   object 
 3   hospital_number        1235 non-null   int64  
 4   rectal_temp            1235 non-null   float64
 5   pulse                  1235 non-null   float64
 6   respiratory_rate       1235 non-null   float64
 7   temp_of_extremities    1196 non-null   object 
 8   peripheral_pulse       1175 non-null   object 
 9   mucous_membrane        1214 non-null   object 
 10  capillary_refill_time  1229 non-null   object 
 11  pain                   1191 non-null   object 
 12  peristalsis            1215 non-null   object 
 13  abdominal_distention   1212 non-null   object 
 14  nasogastric_tube       1155 non-null   object 
 15  naso

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     824 non-null    int64  
 1   surgery                824 non-null    object 
 2   age                    824 non-null    object 
 3   hospital_number        824 non-null    int64  
 4   rectal_temp            824 non-null    float64
 5   pulse                  824 non-null    float64
 6   respiratory_rate       824 non-null    float64
 7   temp_of_extremities    789 non-null    object 
 8   peripheral_pulse       777 non-null    object 
 9   mucous_membrane        811 non-null    object 
 10  capillary_refill_time  818 non-null    object 
 11  pain                   795 non-null    object 
 12  peristalsis            805 non-null    object 
 13  abdominal_distention   802 non-null    object 
 14  nasogastric_tube       760 non-null    object 
 15  nasoga

### Column remover function:

In [7]:
remover = lambda data, col_name: data.drop(columns=[col_name])

In [8]:
train = remover(train, 'id')
train_labels = train['outcome']
train = remover(train, 'outcome')
test = remover(test, 'id')

In [9]:
train.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,...,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no
1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,...,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no
2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,...,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no
3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,...,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes
4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,...,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes


In [10]:
test.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,less_3_sec,...,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,more_3_sec,...,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,less_3_sec,...,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,less_3_sec,...,distend_large,75.0,81.0,NaN,1.0,yes,1400,0,0,no
4,yes,adult,529840,38.3,50.0,12.0,NaN,normal,bright_pink,less_3_sec,...,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes


In [11]:
numeric_feat = train.select_dtypes(include=['float64', 'int64'])
categ_feat = train.select_dtypes(exclude=['float64', 'int64'])
numeric_feat

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
0,530001,38.1,132.0,24.0,6.5,57.0,8.5,3.4,2209,0,0
1,533836,37.5,88.0,12.0,2.0,33.0,64.0,2.0,2208,0,0
2,529812,38.3,120.0,28.0,3.5,37.0,6.4,3.4,5124,0,0
3,5262541,37.1,72.0,30.0,2.0,53.0,7.0,3.9,2208,0,0
4,5299629,38.0,52.0,48.0,7.0,47.0,7.3,2.6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1230,535246,38.5,129.0,48.0,2.0,57.0,66.0,2.0,2206,0,0
1231,528570,37.5,60.0,50.0,3.0,35.0,6.4,3.6,2209,0,0
1232,529685,37.5,84.0,40.0,3.0,40.0,5.9,7.0,400,0,0
1233,534784,38.1,70.0,16.0,2.0,58.0,74.0,2.0,2209,0,0


In [12]:
categ_feat

,surgery,age,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,rectal_exam_feces,abdomen,abdomo_appearance,surgical_lesion,cp_data
0,yes,adult,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,decreased,distend_small,serosanguious,yes,no
1,yes,adult,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,absent,distend_small,serosanguious,yes,no
2,yes,adult,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,NaN,distend_large,serosanguious,yes,no
3,yes,adult,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,decreased,distend_small,cloudy,yes,yes
4,no,adult,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,normal,normal,cloudy,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,yes,adult,cool,reduced,pale_pink,more_3_sec,depressed,absent,moderate,none,more_1_liter,NaN,distend_large,serosanguious,yes,no
1231,yes,adult,cool,reduced,pale_cyanotic,less_3_sec,mild_pain,hypomotile,slight,slight,none,decreased,distend_small,serosanguious,yes,yes
1232,yes,young,normal,reduced,normal_pink,less_3_sec,mild_pain,hypomotile,slight,slight,none,increased,firm,cloudy,yes,yes
1233,yes,adult,normal,reduced,bright_red,less_3_sec,mild_pain,hypomotile,slight,none,more_1_liter,absent,distend_small,cloudy,yes,no


In [13]:
for cat in categ_feat:
    print(f' {cat} has: {categ_feat[cat].unique()}')

 surgery has: ['yes' 'no']
 age has: ['adult' 'young']
 temp_of_extremities has: ['cool' 'cold' 'normal' 'warm' nan]
 peripheral_pulse has: ['reduced' 'normal' nan 'absent' 'increased']
 mucous_membrane has: ['dark_cyanotic' 'pale_cyanotic' 'pale_pink' 'normal_pink' 'bright_pink'
 'bright_red' nan]
 capillary_refill_time has: ['more_3_sec' 'less_3_sec' nan '3']
 pain has: ['depressed' 'mild_pain' 'extreme_pain' 'alert' 'severe_pain' nan 'slight']
 peristalsis has: ['absent' 'hypomotile' 'normal' 'hypermotile' nan 'distend_small']
 abdominal_distention has: ['slight' 'moderate' 'none' 'severe' nan]
 nasogastric_tube has: ['slight' 'none' 'significant' nan]
 nasogastric_reflux has: ['less_1_liter' 'more_1_liter' 'none' nan 'slight']
 rectal_exam_feces has: ['decreased' 'absent' nan 'normal' 'increased' 'serosanguious']
 abdomen has: ['distend_small' 'distend_large' 'normal' 'firm' nan 'other']
 abdomo_appearance has: ['serosanguious' 'cloudy' 'clear' nan]
 surgical_lesion has: ['yes' 'no

### Missing Values

In [14]:
['temp_of_extremities', 'mucous_membrane', 'pain', 'cp_data']

['temp_of_extremities', 'mucous_membrane', 'pain', 'cp_data']

In [15]:
train.isnull().sum()

surgery                    0
age                        0
hospital_number            0
rectal_temp                0
pulse                      0
respiratory_rate           0
temp_of_extremities       39
peripheral_pulse          60
mucous_membrane           21
capillary_refill_time      6
pain                      44
peristalsis               20
abdominal_distention      23
nasogastric_tube          80
nasogastric_reflux        21
nasogastric_reflux_ph      0
rectal_exam_feces        190
abdomen                  213
packed_cell_volume         0
total_protein              0
abdomo_appearance         48
abdomo_protein             0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

#### Percentage of the missing values for each feature:

In [16]:
(train.isnull().sum() / len(train)) * 100

surgery                   0.000000
age                       0.000000
hospital_number           0.000000
rectal_temp               0.000000
pulse                     0.000000
respiratory_rate          0.000000
temp_of_extremities       3.157895
peripheral_pulse          4.858300
mucous_membrane           1.700405
capillary_refill_time     0.485830
pain                      3.562753
peristalsis               1.619433
abdominal_distention      1.862348
nasogastric_tube          6.477733
nasogastric_reflux        1.700405
nasogastric_reflux_ph     0.000000
rectal_exam_feces        15.384615
abdomen                  17.246964
packed_cell_volume        0.000000
total_protein             0.000000
abdomo_appearance         3.886640
abdomo_protein            0.000000
surgical_lesion           0.000000
lesion_1                  0.000000
lesion_2                  0.000000
lesion_3                  0.000000
cp_data                   0.000000
dtype: float64

### Deal with missing values. 
##### All the missing values belong to categorical features, so I've replaced them with the most frequent ones.

In [17]:
categ_imputer = SimpleImputer(strategy='most_frequent')

In [18]:
train = pd.DataFrame(
    categ_imputer.fit_transform(train), columns=train.columns
    ).astype(train.dtypes.to_dict())

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                1235 non-null   object 
 1   age                    1235 non-null   object 
 2   hospital_number        1235 non-null   int64  
 3   rectal_temp            1235 non-null   float64
 4   pulse                  1235 non-null   float64
 5   respiratory_rate       1235 non-null   float64
 6   temp_of_extremities    1235 non-null   object 
 7   peripheral_pulse       1235 non-null   object 
 8   mucous_membrane        1235 non-null   object 
 9   capillary_refill_time  1235 non-null   object 
 10  pain                   1235 non-null   object 
 11  peristalsis            1235 non-null   object 
 12  abdominal_distention   1235 non-null   object 
 13  nasogastric_tube       1235 non-null   object 
 14  nasogastric_reflux     1235 non-null   object 
 15  naso

In [20]:
train.describe()

,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,1.235000e+03,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000
mean,9.545004e+05,38.202186,79.574089,30.054251,4.382591,49.602429,21.388016,3.290931,3832.496356,14.612146,3.577328
std,1.356403e+06,0.788668,29.108638,16.452066,1.937357,10.535800,26.676453,1.589195,5436.733774,193.705735,88.858953
min,5.213990e+05,35.400000,30.000000,8.000000,1.000000,23.000000,3.500000,0.100000,0.000000,0.000000,0.000000
25%,5.288000e+05,37.800000,53.000000,18.000000,2.000000,43.000000,6.600000,2.000000,2205.000000,0.000000,0.000000
50%,5.297770e+05,38.200000,76.000000,28.000000,4.500000,48.000000,7.500000,3.000000,2209.000000,0.000000,0.000000
75%,5.341450e+05,38.600000,100.000000,36.000000,6.000000,57.000000,9.100000,4.300000,3205.000000,0.000000,0.000000
max,5.305129e+06,40.800000,184.000000,96.000000,7.500000,75.000000,89.000000,10.100000,41110.000000,3112.000000,2209.000000


In [21]:
train.isnull().sum()

surgery                  0
age                      0
hospital_number          0
rectal_temp              0
pulse                    0
respiratory_rate         0
temp_of_extremities      0
peripheral_pulse         0
mucous_membrane          0
capillary_refill_time    0
pain                     0
peristalsis              0
abdominal_distention     0
nasogastric_tube         0
nasogastric_reflux       0
nasogastric_reflux_ph    0
rectal_exam_feces        0
abdomen                  0
packed_cell_volume       0
total_protein            0
abdomo_appearance        0
abdomo_protein           0
surgical_lesion          0
lesion_1                 0
lesion_2                 0
lesion_3                 0
cp_data                  0
dtype: int64

In [22]:
oh_yes = train

#### Imputation is completed!

### Now it's time to encode the categorical features!

## Encoding

#### In this part, encoding has been done with respect to the dataset description in [here](https://archive.ics.uci.edu/dataset/47/horse+colic).

In [23]:
ord_features = ['peripheral_pulse', 'capillary_refill_time', 'peristalsis',
                'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
                'rectal_exam_feces', 'abdomen', 'abdomo_appearance']

In [24]:
categories = [['normal', 'increased', 'reduced', 'absent'],
              ['more_3_sec', '3', 'less_3_sec'],
              ['hypomotile', 'normal', 'hypermotile','absent'],
              ['none', 'slight', 'moderate', 'severe'],
              ['none', 'slight', 'significant'],
              ['none', 'less_1_liter' 'more_1_liter'],
              ['normal', 'increased', 'decreased', 'absent'],
              ['normal', 'other', 'firm', 'distend_small', 'distend_large'],
              ['clear', 'cloudy', 'serosanguious'],
              ]

In [25]:
def ordinal_encoder(feature, categories):
    encoder = preprocessing.OrdinalEncoder(categories=categories, handle_unknown='use_encoded_value', unknown_value=-1)
    encoded = encoder.fit_transform(train[feature].values.reshape((train[feature].shape[0], 1)))
    return encoded

In [26]:
def ordinal_maker():
    feat_num = 0
    for feature in ord_features:
        encoded = ordinal_encoder(feature, [categories[feat_num]])
        feat_num += 1
        train[feature] = encoded
    return train

In [27]:
train = ordinal_maker()

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   surgery                1235 non-null   object 
 1   age                    1235 non-null   object 
 2   hospital_number        1235 non-null   int64  
 3   rectal_temp            1235 non-null   float64
 4   pulse                  1235 non-null   float64
 5   respiratory_rate       1235 non-null   float64
 6   temp_of_extremities    1235 non-null   object 
 7   peripheral_pulse       1235 non-null   float64
 8   mucous_membrane        1235 non-null   object 
 9   capillary_refill_time  1235 non-null   float64
 10  pain                   1235 non-null   object 
 11  peristalsis            1235 non-null   float64
 12  abdominal_distention   1235 non-null   float64
 13  nasogastric_tube       1235 non-null   float64
 14  nasogastric_reflux     1235 non-null   float64
 15  naso

### It's time to One-hot encoding!

In [29]:
nominal_cats = train.select_dtypes(include=['object'])
nominal_cats

,surgery,age,temp_of_extremities,mucous_membrane,pain,surgical_lesion,cp_data
0,yes,adult,cool,dark_cyanotic,depressed,yes,no
1,yes,adult,cool,pale_cyanotic,mild_pain,yes,no
2,yes,adult,cool,pale_pink,extreme_pain,yes,no
3,yes,adult,cold,pale_pink,mild_pain,yes,yes
4,no,adult,normal,normal_pink,alert,no,yes
...,...,...,...,...,...,...,...
1230,yes,adult,cool,pale_pink,depressed,yes,no
1231,yes,adult,cool,pale_cyanotic,mild_pain,yes,yes
1232,yes,young,normal,normal_pink,mild_pain,yes,yes
1233,yes,adult,normal,bright_red,mild_pain,yes,no


In [30]:
# def oh_encoder(nom_cats):
#     ohe = preprocessing.OneHotEncoder(sparse_output=False)
#     data = ohe.fit_transform(nom_cats)
#     return data, ohe

In [31]:
ohe = preprocessing.OneHotEncoder(sparse_output=False)
transformed = ohe.fit_transform(nominal_cats)

### Handling OneHotEncoder:
#### Convert the numpy.ndarray to pandas dataframe without any problem:

In [32]:
n_iter = len(transformed) // 5
columns_encoded = ohe.get_feature_names_out(nominal_cats.columns)
full_df = pd.DataFrame()
for i in range(n_iter):
    index = i * 5
    portion = pd.DataFrame(transformed[index:index+5], columns=columns_encoded)
    full_df = pd.concat([full_df, portion])


In [33]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1235 entries, 0 to 4
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   surgery_no                     1235 non-null   float64
 1   surgery_yes                    1235 non-null   float64
 2   age_adult                      1235 non-null   float64
 3   age_young                      1235 non-null   float64
 4   temp_of_extremities_cold       1235 non-null   float64
 5   temp_of_extremities_cool       1235 non-null   float64
 6   temp_of_extremities_normal     1235 non-null   float64
 7   temp_of_extremities_warm       1235 non-null   float64
 8   mucous_membrane_bright_pink    1235 non-null   float64
 9   mucous_membrane_bright_red     1235 non-null   float64
 10  mucous_membrane_dark_cyanotic  1235 non-null   float64
 11  mucous_membrane_normal_pink    1235 non-null   float64
 12  mucous_membrane_pale_cyanotic  1235 non-null   float64
 

In [34]:
len(full_df)

1235

In [35]:
id_series = pd.Series([i for i in range(len(full_df))], name='id')

In [36]:
fitted_full_df = full_df.reset_index(drop=True)
fitted_train = train.reset_index(drop=True)

In [37]:
fitted_fuck = pd.concat([train, fitted_full_df])

In [38]:
fitted_fuck.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,pain_alert,pain_depressed,pain_extreme_pain,pain_mild_pain,pain_severe_pain,pain_slight,surgical_lesion_no,surgical_lesion_yes,cp_data_no,cp_data_yes
0,yes,adult,530001.0,38.1,132.0,24.0,cool,2.0,dark_cyanotic,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,yes,adult,533836.0,37.5,88.0,12.0,cool,0.0,pale_cyanotic,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,yes,adult,529812.0,38.3,120.0,28.0,cool,2.0,pale_pink,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,yes,adult,5262541.0,37.1,72.0,30.0,cold,2.0,pale_pink,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no,adult,5299629.0,38.0,52.0,48.0,normal,0.0,normal_pink,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# hopeInFuck

In [40]:
# hopeInFuck

In [41]:
id_series.shape

(1235,)

In [42]:
full_df_withID = full_df.insert(loc=0, column='id', value=[i for i in range(len(full_df))])

In [43]:
fuck = pd.DataFrame(full_df_withID)

In [44]:
fuck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [45]:
range_ = [i for i in range(len(full_df))]

In [46]:
type(range_)

list

In [47]:
prev_train = train

In [48]:
# train = pd.concat([train.select_dtypes(exclude=['object']), full_df], axis=1)
# train = train.select_dtypes(exclude=['object']).append(full_df)

In [49]:
# combineTest = prev_train.select_dtypes(exclude='object').combine(full_df)

In [50]:
train.select_dtypes(exclude=['object']).head()

,hospital_number,rectal_temp,pulse,respiratory_rate,peripheral_pulse,capillary_refill_time,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,lesion_1,lesion_2,lesion_3
0,530001,38.1,132.0,24.0,2.0,0.0,3.0,1.0,1.0,-1.0,6.5,2.0,3.0,57.0,8.5,2.0,3.4,2209,0,0
1,533836,37.5,88.0,12.0,0.0,0.0,3.0,2.0,0.0,-1.0,2.0,3.0,3.0,33.0,64.0,2.0,2.0,2208,0,0
2,529812,38.3,120.0,28.0,2.0,2.0,0.0,2.0,1.0,0.0,3.5,3.0,4.0,37.0,6.4,2.0,3.4,5124,0,0
3,5262541,37.1,72.0,30.0,2.0,0.0,0.0,2.0,1.0,-1.0,2.0,2.0,3.0,53.0,7.0,1.0,3.9,2208,0,0
4,5299629,38.0,52.0,48.0,0.0,2.0,0.0,0.0,1.0,-1.0,7.0,0.0,0.0,47.0,7.3,1.0,2.6,0,0,0


In [51]:
prev_train.select_dtypes(exclude='object').head()

,hospital_number,rectal_temp,pulse,respiratory_rate,peripheral_pulse,capillary_refill_time,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,lesion_1,lesion_2,lesion_3
0,530001,38.1,132.0,24.0,2.0,0.0,3.0,1.0,1.0,-1.0,6.5,2.0,3.0,57.0,8.5,2.0,3.4,2209,0,0
1,533836,37.5,88.0,12.0,0.0,0.0,3.0,2.0,0.0,-1.0,2.0,3.0,3.0,33.0,64.0,2.0,2.0,2208,0,0
2,529812,38.3,120.0,28.0,2.0,2.0,0.0,2.0,1.0,0.0,3.5,3.0,4.0,37.0,6.4,2.0,3.4,5124,0,0
3,5262541,37.1,72.0,30.0,2.0,0.0,0.0,2.0,1.0,-1.0,2.0,2.0,3.0,53.0,7.0,1.0,3.9,2208,0,0
4,5299629,38.0,52.0,48.0,0.0,2.0,0.0,0.0,1.0,-1.0,7.0,0.0,0.0,47.0,7.3,1.0,2.6,0,0,0


In [52]:
# try:
#     mergedWithShit = shit.merge(transformed, how='outer')
#     print("Hell Yeah.")
# except MergeError:
#     print('Fuck + Shit.')

In [53]:
strange = prev_train + full_df

In [54]:
strange

,abdomen,abdominal_distention,abdomo_appearance,abdomo_protein,age,age_adult,age_young,capillary_refill_time,cp_data,cp_data_no,...,surgery_yes,surgical_lesion,surgical_lesion_no,surgical_lesion_yes,temp_of_extremities,temp_of_extremities_cold,temp_of_extremities_cool,temp_of_extremities_normal,temp_of_extremities_warm,total_protein
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Test

In [55]:
# from sklearn.compose import ColumnTransformer

In [56]:
# ct = ColumnTransformer(["OrdinalEncoding", ])

In [57]:
# ord_features = ['peripheral_pulse', 'capillary_refill_time', 'peristalsis',
#                 'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
#                 'rectal_exam_feces', 'abdomen', 'abdomo_appearance']
# categories = [['normal', 'increased', 'reduced', 'absent'],
#               ['more_3_sec', '3', 'less_3_sec'],
#               ['hypomotile', 'normal', 'hypermotile','absent'],
#               ['none', 'slight', 'moderate', 'severe'],
#               ['none', 'slight', 'significant'],
#               ['none', 'less_1_liter' 'more_1_liter'],
#               ['normal', 'increased', 'decreased', 'absent'],
#               ['normal', 'other', 'firm', 'distend_small', 'distend_large'],
#               ['clear', 'cloudy', 'serosanguious'],
#               ]

num_feat = oh_yes.select_dtypes(include=['float64', 'int64'])

ordinal_categories = {
    'peripheral_pulse': ['normal', 'increased', 'reduced', 'absent'],
    'capillary_refill_time': ['more_3_sec', '3', 'less_3_sec'],
    'peristalsis': ['hypomotile', 'normal', 'hypermotile','absent'],
    'abdominal_distention': ['none', 'slight', 'moderate', 'severe'],
    'nasogastric_tube': ['none', 'slight', 'significant'],
    'nasogastric_reflux': ['none', 'less_1_liter' 'more_1_liter'],
    'rectal_exam_feces': ['normal', 'increased', 'decreased', 'absent'],
    'abdomen': ['normal', 'other', 'firm', 'distend_small', 'distend_large'],
    'abdomo_appearance': ['clear', 'cloudy', 'serosanguious']
}

nominal_cats = ['temp_of_extremities', 'mucous_membrane', 'pain', 'cp_data']


In [58]:
num_feat_col = list(num_feat.columns)

In [59]:
oh_yes.head()

,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,...,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,yes,adult,530001,38.1,132.0,24.0,cool,2.0,dark_cyanotic,0.0,...,3.0,57.0,8.5,2.0,3.4,yes,2209,0,0,no
1,yes,adult,533836,37.5,88.0,12.0,cool,0.0,pale_cyanotic,0.0,...,3.0,33.0,64.0,2.0,2.0,yes,2208,0,0,no
2,yes,adult,529812,38.3,120.0,28.0,cool,2.0,pale_pink,2.0,...,4.0,37.0,6.4,2.0,3.4,yes,5124,0,0,no
3,yes,adult,5262541,37.1,72.0,30.0,cold,2.0,pale_pink,0.0,...,3.0,53.0,7.0,1.0,3.9,yes,2208,0,0,yes
4,no,adult,5299629,38.0,52.0,48.0,normal,0.0,normal_pink,2.0,...,0.0,47.0,7.3,1.0,2.6,no,0,0,0,yes


In [60]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Assuming you have a dataset X with both numerical and categorical features
# Replace 'ordinal_features' and 'nominal_features' with your actual feature names

# Specify the ordinal categories for each ordinal feature
# ordinal_categories = {
#     'ordinal_feature_1': ['low', 'medium', 'high'],
#     'ordinal_feature_2': ['small', 'medium', 'large']
# }

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_feat_col),  # numerical features
        ('ord', OrdinalEncoder(categories=[ordinal_categories[f] for f in list(ordinal_categories.keys())]), list(ordinal_categories.keys())),  # ordinal features
        ('nom', OneHotEncoder(), nominal_cats)  # nominal features
    ])

# Create a pipeline with the column transformer
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Fit and transform the data
X_transformed = pipeline.fit_transform(oh_yes)

ValueError: could not convert string to float: 'normal'

In [ ]:
dict_ = {'ali': 2,
             'mamad': 3}

print(list(dict_.keys()))

['ali', 'mamad']
